In [2]:
import shutil                            
import glob                             
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pandas as pd
import sklearn
from IPython.display import display

In [3]:
from RNASeq_prep_n_RFOptimization import *

In [4]:
os.getcwd()                            
root_path = "/home/io3247/GenFeatSel"
met_path = root_path + "/genomic_blca/genomic_blca_met"
met_content = next(os.walk(met_path))[1]
try:
    met_content.remove(".ipynb_checkpoints")  
except:
    pass
#print(met_content)
print(len(met_content))

80


In [5]:
# Create a list containing path to all "metastasis" rna_seq files.
met_list = []
for barc_folder in met_content:
    barc_folder_path = met_path + "/" + barc_folder
    barc_folder_content = next(os.walk(barc_folder_path))[1]
    met_folder = barc_folder_content[0]
    met_folder_path = barc_folder_path + "/" + met_folder
    met_extension = "/*.tsv"
    rna_seq_files = glob.glob(met_folder_path + met_extension)     
    met_list.append(rna_seq_files[0])
    
#print(met_list)
print(len(met_list))

80


In [6]:
no_met_path = root_path + "/genomic_blca/genomic_blca_no-met"
no_met_content = next(os.walk(no_met_path))[1]  
try:
    no_met_content.remove(".ipynb_checkpoints")  
except:
    pass
#print(no_met_content)
print(len(no_met_content))

80


In [7]:
# Create a list containing path to all "no-metastasis" rna_seq files.
no_met_list = []
for barc_folder in no_met_content:
    barc_folder_path = no_met_path + "/" + barc_folder
    barc_folder_content = next(os.walk(barc_folder_path))[1]
    no_met_folder = barc_folder_content[0]
    no_met_folder_path = barc_folder_path + "/" + no_met_folder
    no_met_extension = "/*.tsv"
    rna_seq_files = glob.glob(no_met_folder_path + no_met_extension)
    no_met_list.append(rna_seq_files[0])
    
#print(no_met_list)
print(len(no_met_list))

80


In [8]:
# Multiple categories data pre-processing pipeline
# Combine tpm_objs of all categories of samples cosidered into a sigle list before further pre-processing
met_tpm_objs = all_tpm_objs_list(met_list, metastasis=True)
met_objs = met_tpm_objs[0]
met_labels = met_tpm_objs[1]
no_met_tpm_objs = all_tpm_objs_list(no_met_list)
no_met_objs = no_met_tpm_objs[0]
no_met_labels = no_met_tpm_objs[1]
all_cat_tpm_objs = met_objs + no_met_objs

# Merge each rna-seq df into one full df
all_cat_full_df = merge_dataframe(all_cat_tpm_objs)
all_cat_full_df

# Carry out further pre-processing- float32 type conversion, 0 values filter, variance filter, and transposition on df, log transformation
prep_df = further_preprocessing(all_cat_full_df, save_filename_dot_csv = "preprocessed_BLCA_tpm.csv", apply_var_filter = True)
prep_df.shape

There are 0 NaN values in dataframe.
Samples values converted to float32
There are 0 NaN values in dataframe after log transformation
There are 463629 Inf values in data after log transformation
0
NaN and Inf values replaced with 0.
Log transformation complete.


/.autofs/tools/spack/var/spack/environments/tumor-22092001/.spack-env/view/lib/python3.8/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log10
  result = func(self.values, **kwargs)


Genes with 0 value in greter than 80% of samples removed.
Dataframe sorted by variance. Top 10,000 genes selected
Dataframe transposed: Genes are now arranged in columns, and samples in rows
Would you like to save transposed, preprocessed full fpkm dataframe? (y/n)n


(160, 10000)

In [9]:
# Create a "metastasis" label column for each category (metastasis / no metastasis)
prep_df.loc[met_labels, "metastasis"] = 1
prep_df.loc[no_met_labels, "metastasis"] = 0
# Convert "metastasis" column to type int
prep_df["metastasis"] = prep_df["metastasis"].astype("int")
prep_df.head(20)

gene_name,KRT6A,SPINK1,KRT5,KRT20,GSTM1,RPS4Y1,HMGCS2,KRT13,DSG3,KRT14,...,SEMA6C,TRIM14,RNF19B,G6PD,SPART,CAT,TMEM65,TNFRSF10A,TMTC2,metastasis
0,,,,,,,,,,,,,,,,,,,,,
met_patient1,2.485847,0.468406,3.304415,2.314423,2.134161,2.771527,1.186009,3.853465,1.490822,1.525784,...,1.059692,1.523922,1.762159,1.778139,1.049319,1.775893,0.994396,1.488546,1.190326,1
met_patient2,3.770939,-0.588212,3.826497,-0.611189,0.199920,-1.266001,-0.863597,0.574922,1.556667,3.864502,...,0.627601,1.249193,1.821425,1.486739,1.187772,2.167517,1.162418,1.298386,0.722782,1
met_patient3,3.994303,0.708030,4.078915,0.324077,0.371012,2.030510,0.087568,1.213881,2.546487,4.447619,...,0.606231,1.454688,1.798896,2.091919,0.926867,1.780687,1.267814,1.325008,0.331529,1
met_patient4,-0.325139,2.241668,2.447783,1.007253,-1.037157,2.448398,2.709300,3.093846,0.222066,-0.467246,...,0.349627,1.470653,1.794031,2.121171,1.503799,2.153338,0.531185,0.917736,0.722206,1
met_patient5,3.101205,-0.693575,3.444901,-0.966174,1.443889,-0.173731,0.415991,1.454945,1.997646,3.041708,...,0.865187,1.732806,2.035776,1.774489,1.292398,1.829720,1.389975,0.898215,0.634024,1
met_patient6,2.855980,0.787723,3.565637,0.719679,2.169149,2.559360,0.508664,4.163777,2.718144,3.064817,...,0.912817,1.562614,1.905865,1.897888,1.176277,1.564493,1.159456,1.589704,1.295030,1
met_patient7,-0.035269,2.674077,1.923787,3.127244,-1.235824,2.743984,2.487469,2.634731,-1.406714,1.325243,...,0.487873,1.488315,1.648729,1.913074,1.545804,2.150972,0.809176,1.157566,1.140637,1
met_patient8,3.153689,1.715438,3.386103,2.503668,2.654869,-0.749824,0.985624,3.207443,1.572401,2.394732,...,1.181726,1.607846,1.720087,1.964063,1.360141,1.793545,1.140722,1.396809,1.027810,1
met_patient9,-1.369572,3.800762,-0.177309,3.007730,2.531929,2.315384,1.230280,3.240573,-1.950782,1.116309,...,0.708106,1.004085,1.750707,1.518511,1.423754,1.696444,0.753154,0.733927,1.140524,1


In [10]:
# Seperate independent variables
X = prep_df.drop(labels = ["metastasis"], axis = 1)
print(X.shape)
# Extract dependent variable
Y = prep_df["metastasis"]
print(Y.shape)

(160, 10000)
(160,)


In [11]:
# Gene ranking using Random Forest Classifier (Metastasis vs. No-Metastasis)
genes_selection_df = rf_model(X,Y)
genes_selection_df

,gene_name,frequency_in_top_100
0,FKBP6,430
313,ASIC5,382
138,MAPK8IP1,332
280,F11R,310
449,PABPC5,297
53,SLC5A1,288
24,CCDC33,273
534,ONECUT1,260
100,CLC,258
125,ZNF467,256


In [14]:
genes_selection_df = genes_selection_df.reset_index(drop=True)
genes_selection_df

,gene_name,frequency_in_top_100
0,FKBP6,430
1,ASIC5,382
2,MAPK8IP1,332
3,F11R,310
4,PABPC5,297
5,SLC5A1,288
6,CCDC33,273
7,ONECUT1,260
8,CLC,258
9,ZNF467,256


In [15]:
genes_selection_df.to_csv("top_genes_BLCA_tpm.csv")